In [ ]:
!pip install gpustat
!gpustat

In [ ]:
import os
import numpy as np
from tensorflow import keras
import imgaug as ia
from matplotlib import pyplot as plt

from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
INPUT_SIZE = (384, 512)
CLASS_COUNT = 6
DATA_DIR = '/content/gdrive/My Drive/data_source/garbage_classifier/training_data/'
BATCH_SIZE = 4
VALIDATION_SPLIT = 0.1

LOG_PATH = '/content/gdrive/My Drive/data_source/garbage_classifier/pretrained_resnet_50_log/'
MODEL_ARCHIVE_PATH = '/content/gdrive/My Drive/data_source/garbage_classifier/pretrained_resnet_50_models/'

In [ ]:
def image_preprocessing(images):
    sometimes = lambda x: ia.augmenters.Sometimes(0.5, x)
    augmentation_sequence = ia.augmenters.Sequential([
        sometimes(ia.augmenters.AdditiveGaussianNoise(loc=(0.0, 0.1))),
        sometimes(ia.augmenters.Add(-0.1, 0.1))
    ])
    return augmentation_sequence(images=images)

data_generator = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=image_preprocessing,
    validation_split=VALIDATION_SPLIT
)
flow_args = {
    'directory': DATA_DIR,
    'target_size': INPUT_SIZE,
    'color_mode': 'rgb',
    'batch_size': BATCH_SIZE,
    'class_mode': 'categorical',
    'save_format': 'jpeg'
}

training_data_generator = data_generator.flow_from_directory(
    **flow_args,
    subset='training'
)
validation_data_generator = data_generator.flow_from_directory(
    **flow_args,
    subset='validation'
)
print(training_data_generator.class_indices)

In [ ]:
base_model = keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = False
layer = base_model.output
layer = keras.layers.GlobalAveragePooling2D()(layer)
layer = keras.layers.Dense(2048, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4))(layer)
layer = keras.layers.Dense(CLASS_COUNT, activation='softmax')(layer)
model = keras.models.Model(inputs=base_model.input, outputs=layer)
model.compile(
    optimizer=keras.optimizers.Adam(), 
    loss=keras.losses.categorical_crossentropy, 
    metrics=['accuracy']
)

In [ ]:
model.fit(
    training_data_generator,
    validation_data=validation_data_generator,
    epochs=32,
    steps_per_epoch=512,
    validation_steps=64,
    callbacks=[
        keras.callbacks.LearningRateScheduler(lambda ei, lr: lr / 2.0 if ei > 0 and (ei / 8).is_integer() else lr),
        keras.callbacks.TensorBoard(log_dir=LOG_PATH, update_freq=32),
        keras.callbacks.ModelCheckpoint(os.path.join(MODEL_ARCHIVE_PATH, 'model_checkpoint_{epoch:02d}.hdf5'), period=4)
    ]
)